부산 동백전

In [224]:
import pandas as pd
import numpy as np
import func01
import openpyxl

In [225]:
# 동백전 데이터
file_Data = r'./2023.09_집계구 단위 업종별 지역화폐 거래 데이터.csv'
dataDF = pd.read_csv(file_Data, encoding='euc-kr',header=None,dtype=str)

부산 인구 데이터 처리 <hr>

In [226]:
file_busan = './행정구역_시군구_별__성별_인구수_20250122101039.csv'
busanDF = pd.read_csv(file_busan)
busanDF.columns = busanDF.iloc[0]
busanDF.drop(0,inplace=True)
busanDF[['총인구수 (명)','남자인구수 (명)','여자인구수 (명)']] = busanDF[['총인구수 (명)','남자인구수 (명)','여자인구수 (명)']].astype(int)

In [227]:
busan_total = busanDF['총인구수 (명)'].sum()

busan_total = "{:,}".format(busan_total)
print(f'부산 전체 인구수 : {busan_total}명')

부산 전체 인구수 : 6,601,672명


In [228]:
dataDF4 = busanDF[busanDF['행정구역(시군구)별'].isin(['연제구', '영도구', '중구', '해운대구'])]
display(dataDF4)

,행정구역(시군구)별,총인구수 (명),남자인구수 (명),여자인구수 (명)
2,중구,38956,18873,20083
5,영도구,107003,52565,54438
10,해운대구,382115,183135,198980
14,연제구,202884,97043,105841


동백전 파일 전처리 <hr>

In [229]:
dataDF.columns = ['기준일자','기준시간','시도명','시군구명','행정동명','집계구코드','집계구중심 경도','집계구중심 위도',
                  '카드사 명칭','업종코드','업종명칭','생년','성별','거래횟수','총 거래금액']

dataDF = dataDF.drop(0)   # 기존 칼럼명 삭제 
#             안쓰는 데이터 없애기
dataDF = dataDF.drop(columns=['기준시간','시도명','집계구코드','집계구중심 경도','집계구중심 위도','업종코드'])
# 맡은 구만 저장하기
dataDF = dataDF[dataDF['시군구명'].isin(['연제구', '영도구', '중구', '해운대구'])]
dataDF

,기준일자,시군구명,행정동명,카드사 명칭,업종명칭,생년,성별,거래횟수,총 거래금액
5345,20230901,연제구,거제1동,부산,편 의 점,1994,F,1,700
5346,20230901,연제구,거제1동,코나,GS 가스충전소,1959,F,1,30454
5347,20230901,연제구,거제1동,코나,서양음식,1959,M,1,1500
5348,20230901,연제구,거제1동,코나,서양음식,1980,F,1,1300
5349,20230901,연제구,거제1동,부산,슈퍼마켓,1985,M,1,6050
...,...,...,...,...,...,...,...,...,...
4756455,20230930,해운대구,중1동,부산,일반한식,1982,F,1,36000
4756456,20230930,해운대구,중1동,코나,기타숙박업,1983,F,1,30000
4756457,20230930,해운대구,중1동,코나,일반한식,1959,M,1,26000
4756458,20230930,해운대구,중1동,코나,일반한식,1968,F,1,54000


업종명 카테고리

In [230]:
replace_list = ['레져용품수리', '골프연습장', '문화취미기타', '레져업소', '기타레져업소', '스포츠·레져용품' 
                '골프용품 전문점', '볼 링 장', '수 영 장', '테니스장', '당 구 장','헬스클럽']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '스포츠')

In [231]:
replace_list = ['세탁소', '보관및 창고업', '애완동물', '소프트웨어', '기타농업관련', '내열기기', '종합레져타운', '여객선',
                '사우나', '주차장', '비영리/대상', 'DP&E', '기타 전문점',  '통신기기', '기타가구', '목재·석재·철물', '기타1', '기타4', '수족관',
                '세차장', '티켓','기타3','혼수전문점', '비영리/비대상']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '기타')

In [232]:
replace_list = ['기타용역서비스','법률회계서비스(개인)', '영화관', '안마/스포츠마사지', '화물운송','사무·통신기기수리', '기타수리서비스', '인테리어전문',
                '관광여행', '법률회계서비스(법인)', '기타대인서비스', '신변잡화수리', '부동산 분양', '부동산 중개·임대', '종합용역','보일러·펌프·샷시',
                '침구·수예점', '화원', '화랑', '피아노대리점', '가정용품수리']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '서비스업')

In [233]:
replace_list = ['기타숙박업', '1급 호텔', '2급 호텔', '콘도']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '숙박업')

In [234]:
replace_list = ['가전제품', '컴퓨터', '조명기구', '철제가구', '악기점', '사무서비스', '기계공구', '안경', '완구점', '기타광학제품',
                '비료.사료.종자', '기념품점', '주방용식기', '화방·표구점', '페인트', '농기계', '과학기자재', '건축용 요업제품',
                '미곡상', '기타건축자재', '기타전기제품', '시계', '카메라', '귀금속', '기타잡화', '사무용 OA기기', 
                '문구용품', '민예·공예품', 'DVD·음반·테이프 판매', '기타 사무용품','주방용구','일반가구', '카페트.커튼.천막.지물',
                '정수기', '유리', '냉열기기']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '잡화')

In [235]:
replace_list = ['일반서적', '출판 및 인쇄물', '기타서적문구', '전문서적', '정기간행물' ,'서적']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '서적')

In [236]:
replace_list = ['기능학원', '유학원', '예·체능계학원', '학습지 교육', '컴퓨터학원', '외국어학원', '기타 교육기관', '유치원', '보습학원',
                '학원', '독서실']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '교육')

In [237]:
replace_list = ['택시']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '교통')

In [238]:
replace_list = ['의원', '한의원', '제약회사', '슈퍼마켓', '동물병원', '병원(응급실 운영)', '치과병원', '산후조리원', '치과의원', '의료용품',
                '종합병원', '한약방', '한방병원', '기타의료기관 및 기타의료기기', '약국', '기타건강식품', '건강식품',]

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '의료 및 보건')

In [239]:
replace_list = ['아동의류', '가방', '기타의류', '내의판매', '신발', '단체복', '기타직물', '제화', '양 품 점', '액세서리', '스포츠 의류',
                '옷감·직물', '정장(남성)', '맞춤복점', '단체복']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '의류')

In [240]:
replace_list = ['편 의 점', '대형할인점', '농·축협 직영매장', '농·축·수산품', '기타비영리유통', '농축수산 가공품', 
                '기타유통업', '연 쇄 점', '정육점', '농.축협 직영매장']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '유통업')

In [241]:
replace_list = ['일식·회집', '일반한식', '중국식', '칵테일바', '주류 판매점', '서양음식', '한정식', '갈비전문점', '제과점', '스넥',
                '위탁급식업', '기타음료식품', '홍삼제품', '인삼제품']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '음식')

In [242]:
replace_list = ['주점', '단란주점', '유흥주점', '노 래 방']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '유흥주점')

In [243]:
replace_list = ['미용재료', '미용원', '피부미용실', '화장품', '이용원']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '미용')

In [244]:
replace_list = ['쌍용S-OIL', 'SK주유소', '현대정유(오일뱅크)', 'GS주유소', 'LPG 취급점', 'SK가스충전소', '쌍용S-OIL 가스충전소', '기타연료',
                '주유소', '유류판매', 'GS 가스충전소']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '주유소')

In [245]:
replace_list = ['자동차부품', '자동차시트·타이어', '윤활유 전문 판매', '견인서비스', '자동차정비', '가타자동차서비스', '자동차서비스',
                '렌터카', '국산신차 직영부품·정비업소', '이륜차판매', '카인테리어']

dataDF['업종명칭'] = dataDF['업종명칭'].mask(dataDF['업종명칭'].isin(replace_list), '자동차관련업')

In [246]:
dataDF['업종명칭'].unique()

array(['유통업', '주유소', '음식', '의료 및 보건', '미용', '서비스업', '기타', '교육', '서적',
       '교통', '잡화', '자동차관련업', '스포츠', '유흥주점', '의류', '스포츠·레져용품', '골프용품 전문점',
       '숙박업'], dtype=object)

In [247]:
# 데이터 요약 정보
print(dataDF)
dataDF.info()

             기준일자  시군구명  행정동명 카드사 명칭     업종명칭    생년 성별 거래횟수 총 거래금액
5345     20230901   연제구  거제1동     부산      유통업  1994  F    1    700
5346     20230901   연제구  거제1동     코나      주유소  1959  F    1  30454
5347     20230901   연제구  거제1동     코나       음식  1959  M    1   1500
5348     20230901   연제구  거제1동     코나       음식  1980  F    1   1300
5349     20230901   연제구  거제1동     부산  의료 및 보건  1985  M    1   6050
...           ...   ...   ...    ...      ...   ... ..  ...    ...
4756455  20230930  해운대구   중1동     부산       음식  1982  F    1  36000
4756456  20230930  해운대구   중1동     코나      숙박업  1983  F    1  30000
4756457  20230930  해운대구   중1동     코나       음식  1959  M    1  26000
4756458  20230930  해운대구   중1동     코나       음식  1968  F    1  54000
4756459  20230930  해운대구   중1동     코나       음식  1968  F    1   1500

[1069313 rows x 9 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 1069313 entries, 5345 to 4756459
Data columns (total 9 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  -------

In [248]:
# 사용할 숫자 데이터 object타입에서 int타입으로 변경
dataDF[['생년','총 거래금액']] = dataDF[['생년','총 거래금액']].astype(int)
print(dataDF.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1069313 entries, 5345 to 4756459
Data columns (total 9 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   기준일자    1069313 non-null  object
 1   시군구명    1069313 non-null  object
 2   행정동명    1069313 non-null  object
 3   카드사 명칭  1069313 non-null  object
 4   업종명칭    1069313 non-null  object
 5   생년      1069313 non-null  int64 
 6   성별      1069313 non-null  object
 7   거래횟수    1069313 non-null  object
 8   총 거래금액  1069313 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 81.6+ MB
None


In [249]:
# 생년을 나이로 변경
dataDF.rename(columns={'생년':'나이'},inplace=True)
dataDF['나이'] = 2025 - dataDF['나이']

# 음수 나이를 결측값으로 처리
dataDF['나이'] = dataDF['나이'].where(dataDF['나이'] >= 0, np.nan)

# 결측값 확인
print(f"결측값 처리 후 나이 컬럼에서 NaN 개수: {dataDF['나이'].isnull().sum()}")

# 결측값이 있는 행 제거
dataDF = dataDF.dropna(subset=['나이'])

# 결측값 확인
print(f"결측값 처리 후 나이 컬럼에서 NaN 개수: {dataDF['나이'].isnull().sum()}")

결측값 처리 후 나이 컬럼에서 NaN 개수: 18
결측값 처리 후 나이 컬럼에서 NaN 개수: 0


In [250]:
# 나이를 연령대로 변경
def ages_group(age):
    if age >= 60:
        return '60대이상'
    elif age >= 50:
        return '50대'
    elif age >= 40:
        return '40대'
    elif age >= 30:
        return '30대'
    elif age >= 20:
        return '20대'
    else: '10대'

dataDF = dataDF.copy()
dataDF['연령대'] = dataDF['나이'].apply(ages_group)

print(dataDF.info())
dataDF

<class 'pandas.core.frame.DataFrame'>
Index: 1069295 entries, 5345 to 4756459
Data columns (total 10 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   기준일자    1069295 non-null  object 
 1   시군구명    1069295 non-null  object 
 2   행정동명    1069295 non-null  object 
 3   카드사 명칭  1069295 non-null  object 
 4   업종명칭    1069295 non-null  object 
 5   나이      1069295 non-null  float64
 6   성별      1069295 non-null  object 
 7   거래횟수    1069295 non-null  object 
 8   총 거래금액  1069295 non-null  int64  
 9   연령대     1067139 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 89.7+ MB
None


,기준일자,시군구명,행정동명,카드사 명칭,업종명칭,나이,성별,거래횟수,총 거래금액,연령대
5345,20230901,연제구,거제1동,부산,유통업,31.0,F,1,700,30대
5346,20230901,연제구,거제1동,코나,주유소,66.0,F,1,30454,60대이상
5347,20230901,연제구,거제1동,코나,음식,66.0,M,1,1500,60대이상
5348,20230901,연제구,거제1동,코나,음식,45.0,F,1,1300,40대
5349,20230901,연제구,거제1동,부산,의료 및 보건,40.0,M,1,6050,40대
...,...,...,...,...,...,...,...,...,...,...
4756455,20230930,해운대구,중1동,부산,음식,43.0,F,1,36000,40대
4756456,20230930,해운대구,중1동,코나,숙박업,42.0,F,1,30000,40대
4756457,20230930,해운대구,중1동,코나,음식,66.0,M,1,26000,60대이상
4756458,20230930,해운대구,중1동,코나,음식,57.0,F,1,54000,50대


In [251]:
func01.base_info(dataDF,'부산 동백전')

---[부산 동백전]---

부산 동백전.dtypes > 

기준일자       object
시군구명       object
행정동명       object
카드사 명칭     object
업종명칭       object
나이        float64
성별         object
거래횟수       object
총 거래금액      int64
연령대        object
dtype: object



부산 동백전.index > 
Index([   5345,    5346,    5347,    5348,    5349,    5350,    5351,    5352,
          5353,    5354,
       ...
       4756450, 4756451, 4756452, 4756453, 4756454, 4756455, 4756456, 4756457,
       4756458, 4756459],
      dtype='int64', length=1069295)



부산 동백전.columns > 
Index(['기준일자', '시군구명', '행정동명', '카드사 명칭', '업종명칭', '나이', '성별', '거래횟수', '총 거래금액',
       '연령대'],
      dtype='object')



부산 동백전.ndim > 
2



부산 동백전.shape > 
(1069295, 10)



부산 동백전.values > 
 [['20230901' '연제구' '거제1동' ... '1' 700 '30대']
 ['20230901' '연제구' '거제1동' ... '1' 30454 '60대이상']
 ['20230901' '연제구' '거제1동' ... '1' 1500 '60대이상']
 ...
 ['20230930' '해운대구' '중1동' ... '1' 26000 '60대이상']
 ['20230930' '해운대구' '중1동' ... '1' 54000 '50대']
 ['20230930' '해운대구' '중1동' ... '1' 1500 '50

In [252]:
func01.summary(dataDF, '부산 동백전')

<class 'pandas.core.frame.DataFrame'>
Index: 1069295 entries, 5345 to 4756459
Data columns (total 10 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   기준일자    1069295 non-null  object 
 1   시군구명    1069295 non-null  object 
 2   행정동명    1069295 non-null  object 
 3   카드사 명칭  1069295 non-null  object 
 4   업종명칭    1069295 non-null  object 
 5   나이      1069295 non-null  float64
 6   성별      1069295 non-null  object 
 7   거래횟수    1069295 non-null  object 
 8   총 거래금액  1069295 non-null  int64  
 9   연령대     1067139 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 89.7+ MB
--[부산 동백전]--

           기준일자 시군구명  행정동명 카드사 명칭 업종명칭    나이 성별 거래횟수  총 거래금액    연령대
5345  20230901  연제구  거제1동     부산  유통업  31.0  F    1     700    30대
5346  20230901  연제구  거제1동     코나  주유소  66.0  F    1   30454  60대이상
--[부산 동백전]--

              기준일자  시군구명 행정동명 카드사 명칭 업종명칭    나이 성별 거래횟수  총 거래금액  연령대
4756458  20230930  해운대구  중1동     코나   음식  57.0  F    1   54

연제구 영도구 중구 해운대구 <hr>
- 각 구별 토탈 금액 추산

In [253]:
# 각 구별 총 거래 금액 토탈
total_data = dataDF.groupby('시군구명')['총 거래금액'].sum().reset_index()

# 각 구 업종별 총 거래금액
top_data = dataDF.groupby(['시군구명', '업종명칭'])['총 거래금액'].sum().reset_index()

# 결과 출력
print(f"각 구별 총 거래 금액 합계: {total_data}")

각 구별 총 거래 금액 합계:    시군구명       총 거래금액
0   연제구   7226155150
1   영도구   1862554692
2    중구   2506987061
3  해운대구  11968418906


In [254]:
# 내림차순
top_data = top_data.sort_values(['시군구명', '총 거래금액'], ascending=False)
print(top_data)

    시군구명      업종명칭      총 거래금액
65  해운대구        음식  3900365679
66  해운대구   의료 및 보건  2326561427
54  해운대구        교육  1992643318
63  해운대구       유통업  1553223855
70  해운대구       주유소   394161536
..   ...       ...         ...
14   연제구        의류    24391600
6    연제구        서적    22771053
9    연제구  스포츠·레져용품    15594800
7    연제구       숙박업     3747580
0    연제구  골프용품 전문점     2681000

[71 rows x 3 columns]


In [255]:
# 오름차순
bottom_data = top_data.sort_values(['시군구명', '총 거래금액'])
print(bottom_data)

    시군구명      업종명칭      총 거래금액
0    연제구  골프용품 전문점     2681000
7    연제구       숙박업     3747580
9    연제구  스포츠·레져용품    15594800
6    연제구        서적    22771053
14   연제구        의류    24391600
..   ...       ...         ...
70  해운대구       주유소   394161536
63  해운대구       유통업  1553223855
54  해운대구        교육  1992643318
66  해운대구   의료 및 보건  2326561427
65  해운대구        음식  3900365679

[71 rows x 3 columns]


In [256]:
# 각 구별 가장 많이, 적게 사용된 업종
top_name = dataDF[dataDF['업종명칭'].isin(top_data['업종명칭'])].head(5)
print(top_name)
bottom_name = dataDF[dataDF['업종명칭'].isin(bottom_data['업종명칭'])].head(5)
print(bottom_name)

          기준일자 시군구명  행정동명 카드사 명칭     업종명칭    나이 성별 거래횟수  총 거래금액    연령대
5345  20230901  연제구  거제1동     부산      유통업  31.0  F    1     700    30대
5346  20230901  연제구  거제1동     코나      주유소  66.0  F    1   30454  60대이상
5347  20230901  연제구  거제1동     코나       음식  66.0  M    1    1500  60대이상
5348  20230901  연제구  거제1동     코나       음식  45.0  F    1    1300    40대
5349  20230901  연제구  거제1동     부산  의료 및 보건  40.0  M    1    6050    40대
          기준일자 시군구명  행정동명 카드사 명칭     업종명칭    나이 성별 거래횟수  총 거래금액    연령대
5345  20230901  연제구  거제1동     부산      유통업  31.0  F    1     700    30대
5346  20230901  연제구  거제1동     코나      주유소  66.0  F    1   30454  60대이상
5347  20230901  연제구  거제1동     코나       음식  66.0  M    1    1500  60대이상
5348  20230901  연제구  거제1동     코나       음식  45.0  F    1    1300    40대
5349  20230901  연제구  거제1동     부산  의료 및 보건  40.0  M    1    6050    40대


가장 많이 쓴 업종

In [257]:
data1 = top_data[top_data['시군구명'] == '연제구']
print(f'-----연제구----- \n {data1.head(5)}')

data2 = top_data[top_data['시군구명'] == '영도구']
print(f'-----영도구----- \n {data2.head(5)}')

data3 = top_data[top_data['시군구명'] == '중구']
print(f'-----중구----- \n {data3.head(5)}')

data4 = top_data[top_data['시군구명'] == '해운대구']
print(f'-----해운대구----- \n {data4.head(5)}')

-----연제구----- 
    시군구명     업종명칭      총 거래금액
12  연제구       음식  2615732050
13  연제구  의료 및 보건  1495855118
1   연제구       교육  1093344655
10  연제구      유통업   811337797
17  연제구      주유소   290048833
-----영도구----- 
    시군구명     업종명칭     총 거래금액
30  영도구       음식  836913274
31  영도구  의료 및 보건  357692521
28  영도구      유통업  301262794
19  영도구       교육  118922500
22  영도구       미용   40570400
-----중구----- 
    시군구명     업종명칭      총 거래금액
47   중구       음식  1284242353
48   중구  의료 및 보건   437942106
45   중구      유통업   392119515
49   중구       의류    82460430
51   중구       잡화    77319155
-----해운대구----- 
     시군구명     업종명칭      총 거래금액
65  해운대구       음식  3900365679
66  해운대구  의료 및 보건  2326561427
54  해운대구       교육  1992643318
63  해운대구      유통업  1553223855
70  해운대구      주유소   394161536


In [258]:
display(data1.head(5))
display(data2.head(5))
display(data3.head(5))
display(data4.head(5))

,시군구명,업종명칭,총 거래금액
12,연제구,음식,2615732050
13,연제구,의료 및 보건,1495855118
1,연제구,교육,1093344655
10,연제구,유통업,811337797
17,연제구,주유소,290048833


,시군구명,업종명칭,총 거래금액
30,영도구,음식,836913274
31,영도구,의료 및 보건,357692521
28,영도구,유통업,301262794
19,영도구,교육,118922500
22,영도구,미용,40570400


,시군구명,업종명칭,총 거래금액
47,중구,음식,1284242353
48,중구,의료 및 보건,437942106
45,중구,유통업,392119515
49,중구,의류,82460430
51,중구,잡화,77319155


,시군구명,업종명칭,총 거래금액
65,해운대구,음식,3900365679
66,해운대구,의료 및 보건,2326561427
54,해운대구,교육,1992643318
63,해운대구,유통업,1553223855
70,해운대구,주유소,394161536


가장 적게 쓴 업종

In [259]:
data1 = bottom_data[bottom_data['시군구명'] == '연제구']
print(f'-----연제구----- \n {data1.head(5)}')

data2 = bottom_data[bottom_data['시군구명'] == '영도구']
print(f'-----영도구----- \n {data2.head(5)}')

data3 = bottom_data[bottom_data['시군구명'] == '중구']
print(f'-----중구----- \n {data3.head(5)}')

data4 = bottom_data[bottom_data['시군구명'] == '해운대구']
print(f'-----해운대구----- \n {data4.head(5)}')
display(data4)

-----연제구----- 
    시군구명      업종명칭    총 거래금액
0   연제구  골프용품 전문점   2681000
7   연제구       숙박업   3747580
9   연제구  스포츠·레져용품  15594800
6   연제구        서적  22771053
14  연제구        의류  24391600
-----영도구----- 
    시군구명      업종명칭   총 거래금액
24  영도구        서적   534160
18  영도구  골프용품 전문점  1236000
23  영도구      서비스업  2342300
25  영도구       숙박업  2909700
27  영도구  스포츠·레져용품  4610600
-----중구----- 
    시군구명    업종명칭   총 거래금액
42   중구     숙박업  2061650
52   중구     주유소  4603690
37   중구      교통  5602880
40   중구    서비스업  7565313
50   중구  자동차관련업  8081158
-----해운대구----- 
     시군구명      업종명칭    총 거래금액
53  해운대구  골프용품 전문점  11920000
62  해운대구  스포츠·레져용품  15667350
60  해운대구       숙박업  22106349
67  해운대구        의류  41702300
59  해운대구        서적  42165520


,시군구명,업종명칭,총 거래금액
53,해운대구,골프용품 전문점,11920000
62,해운대구,스포츠·레져용품,15667350
60,해운대구,숙박업,22106349
67,해운대구,의류,41702300
59,해운대구,서적,42165520
64,해운대구,유흥주점,45558480
58,해운대구,서비스업,67018640
55,해운대구,교통,89173780
68,해운대구,자동차관련업,140819953
56,해운대구,기타,245154699


In [260]:
display(data1.head(5))
display(data2.head(5))
display(data3.head(5))
display(data4.head(5))

,시군구명,업종명칭,총 거래금액
0,연제구,골프용품 전문점,2681000
7,연제구,숙박업,3747580
9,연제구,스포츠·레져용품,15594800
6,연제구,서적,22771053
14,연제구,의류,24391600


,시군구명,업종명칭,총 거래금액
24,영도구,서적,534160
18,영도구,골프용품 전문점,1236000
23,영도구,서비스업,2342300
25,영도구,숙박업,2909700
27,영도구,스포츠·레져용품,4610600


,시군구명,업종명칭,총 거래금액
42,중구,숙박업,2061650
52,중구,주유소,4603690
37,중구,교통,5602880
40,중구,서비스업,7565313
50,중구,자동차관련업,8081158


,시군구명,업종명칭,총 거래금액
53,해운대구,골프용품 전문점,11920000
62,해운대구,스포츠·레져용품,15667350
60,해운대구,숙박업,22106349
67,해운대구,의류,41702300
59,해운대구,서적,42165520


In [261]:
age_data = dataDF.groupby(['시군구명', '연령대'])['총 거래금액'].sum().reset_index()
age_top = age_data.sort_values(['시군구명', '연령대'], ascending=False)
print(age_top)


    시군구명    연령대      총 거래금액
19  해운대구  60대이상  3026515601
18  해운대구    50대  2993957365
17  해운대구    40대  3512209289
16  해운대구    30대  1795705030
15  해운대구    20대   564415075
14    중구  60대이상   814661830
13    중구    50대   548056874
12    중구    40대   514090224
11    중구    30대   453593395
10    중구    20대   174566558
9    영도구  60대이상   597371128
8    영도구    50대   473180545
7    영도구    40대   445254091
6    영도구    30대   250129274
5    영도구    20대    91811634
4    연제구  60대이상  1866552967
3    연제구    50대  1801400334
2    연제구    40대  2109307590
1    연제구    30대  1091234367
0    연제구    20대   317790852


In [262]:
gu_list = ['연제구', '영도구', '중구', '해운대구']

# 각 구에 대해 연령대 순위 출력
for i in gu_list:
    dataDFa = dataDF[dataDF['시군구명'] == i]
    list = dataDFa['연령대'].value_counts().sort_values(ascending=False)
    
    print(f'{i} 연령대 순위: {list}\n')

연제구 연령대 순위: 연령대
60대이상    90812
40대      83795
50대      82322
30대      58207
20대      20873
Name: count, dtype: int64

영도구 연령대 순위: 연령대
60대이상    31151
50대      25338
40대      22909
30대      15393
20대       7633
Name: count, dtype: int64

중구 연령대 순위: 연령대
60대이상    33359
50대      26575
40대      25018
30대      23017
20대       9707
Name: count, dtype: int64

해운대구 연령대 순위: 연령대
60대이상    139604
50대      128236
40대      124190
30대       86318
20대       32682
Name: count, dtype: int64



In [263]:
def age_chart(data,region):
    dataDF_a = data[data['시군구명'] == region ]
    age_count = dataDF_a['연령대'].value_counts().sort_values(ascending=False)
    print(f"{region}의 연령대별 거래량 순위: \n{age_count}\n")
    
print('1권역','\n')
age_chart(dataDF,'연제구')
age_chart(dataDF,'영도구')
age_chart(dataDF,'중구')
age_chart(dataDF,'해운대구')

1권역 

연제구의 연령대별 거래량 순위: 
연령대
60대이상    90812
40대      83795
50대      82322
30대      58207
20대      20873
Name: count, dtype: int64

영도구의 연령대별 거래량 순위: 
연령대
60대이상    31151
50대      25338
40대      22909
30대      15393
20대       7633
Name: count, dtype: int64

중구의 연령대별 거래량 순위: 
연령대
60대이상    33359
50대      26575
40대      25018
30대      23017
20대       9707
Name: count, dtype: int64

해운대구의 연령대별 거래량 순위: 
연령대
60대이상    139604
50대      128236
40대      124190
30대       86318
20대       32682
Name: count, dtype: int64

